# Code used to generate the data set

In [46]:
import numpy as np
import os
import math
import shutil
# from moviepy.video.io.VideoFileClip import VideoFileClip
# from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

In [49]:
if not os.path.exists('videos'):
    shutil.unpack_archive('videos.zip', 'videos')


In [2]:
#parse metadata
with open("data_unprocessed/metadata.txt") as file:
    lines =  np.array([line.rstrip().split(";") for line in file])


In [3]:
newpath = r'./temp' 
if not os.path.exists(newpath):
    os.makedirs(newpath)

Ideally we would use a python library to download the youtube videos here. Unfortunatly a recent change to Youtubes cypher system has rendered all of these tools non-functional. Since our data set has a small number of long videos, we have opted to download them manually.

Currently the raw videos are not included in the repo, later I will include them in a zip archive if the github file limit allows it

In [6]:
from moviepy.config import FFMPEG_BINARY
import subprocess

def ffmpeg_extract_subclip(
    inputfile, start_time, end_time, outputfile=None, logger="bar"
):
    """Makes a new video file playing video file between two times.

    Parameters
    ----------

    inputfile : str
      Path to the file from which the subclip will be extracted.

    start_time : float
      Moment of the input clip that marks the start of the produced subclip.

    end_time : float
      Moment of the input clip that marks the end of the produced subclip.

    outputfile : str, optional
      Path to the output file. Defaults to
      ``<inputfile_name>SUB<start_time>_<end_time><ext>``.
    """
    if not outputfile:
        name, ext = os.path.splitext(inputfile)
        t1, t2 = [int(1000 * t) for t in [start_time, end_time]]
        outputfile = "%sSUB%d_%d%s" % (name, t1, t2, ext)

    cmd = [
        FFMPEG_BINARY,
        "-y",
        "-ss",
        "%0.2f" % start_time,
        "-i",
        inputfile,
        "-to",
        "%0.2f" % end_time,
        "-map",
        "0",
        "-vcodec",
        "copy",
        "-acodec",
        "copy",
        "-copyts",
        outputfile,
    ]
    subprocess.run(cmd)

In [45]:
links = np.unique(lines.transpose()[4])
sample = lines[2]

i=0
while i < len(lines):
    
    if float(lines[i][5]) == float(lines[i+1][5])  and lines[i][4] == lines[i+1][4]:
        sample = lines[i if lines[i][6]> lines[i+1][6]  else i+1]
        start = float(lines[i][5])
        end = max(float(lines[i][6]), float(lines[i+1][6]))
        
        name = lines[i][0]+"&"+lines[i+1][0]
        i+=2
    else:
        sample = lines[i]
        start = float(sample[5])
        end = float(sample[6])
        name = sample[0]
        i+=1

    idx = np.where(links == sample[4])
    ## If its the first video, remove once all have been downloaded
    if idx[0][0] == 0:
        ffmpeg_extract_subclip("videos/"+str(idx[0][0])+".mp4", start, end, "cropped/" + name+".mp4")
        
       



Create a zip archive of the output so the videos can be commited to github

In [48]:
if os.path.isfile("videos_processed.zip"):
    os.remove("videos_processed.zip")
shutil.make_archive("videos_processed", 'zip', "cropped")

'c:\\Users\\mikeG\\Documents\\school\\cisc-452\\CISC-452-Group-22\\videos_processed.zip'